<a href="https://colab.research.google.com/github/yougoal/leejunghoon/blob/master/dqn2015r3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#참조: https://hunkim.github.io/ml/
#setup
import numpy as np
import tensorflow as tf
import random 
from collections import deque
import gym


env = gym.make('CartPole-v0')

input_size = env.observation_space.shape[0]
output_size = env.action_space.n
dis = 1
#REPLAY_MEMORY =5
class DQN:
    def __init__(self, session, input_size, output_size, name="main"):
        self.session = session
        self.input_size = input_size
        self.output_size = output_size
        self.net_name = name
        self.build_network()
    def build_network(self, h_size=10, l_rate=1e-1):
        with tf.variable_scope(self.net_name):
            self.X = tf.placeholder(tf.float32, [None, self.input_size], name="input_x")
            print("훈련에 사용되는  self._X: St")
            print(self.X)
            self.W1 = tf.get_variable('W1', shape = [input_size, output_size], initializer=tf.contrib.layers.xavier_initializer())
            print("훈련함여한 W1")
            print(self.W1)
            self.Qpred = tf.matmul(self.X, self.W1)
        self.Y = tf.placeholder(shape = [None, self.output_size], dtype = tf.float32)
        self.loss = tf.reduce_mean(tf.square(self.Y - self.Qpred))
        self.train = tf.train.AdamOptimizer(learning_rate=l_rate).minimize(self.loss)
    def predict(self, state):
        x = np.reshape(state, [1, self.input_size])
        return self.session.run(self.Qpred, feed_dict={self.X: x})
    def update(self, x_stack, y_stack):
        return self.session.run([self.loss, self.train], feed_dict={self.X: x_stack, self.Y: y_stack})
def replay_train(mainDQN, targetDQN, train_batch):
    x_stack = np.empty(0).reshape(0, 4)  # STACK 정의
    y_stack = np.empty(0).reshape(0, 2)  # STACK 정의
    for state, action, reward, next_state, done in train_batch:
        Q = mainDQN.predict(state)       
        print("이때 사용된 mainDQN의 W 및 st")
        print(sess.run(mainDQN._W1))
        print("이때 사용된  St")
        print(state)
        print("mainDQN의 예측값 stW 출력")
        print(Q)
        if done:
            Q[0, action] = reward
        else:
            Q[0, action] = reward + dis * np.max(targetDQN.predict(next_state))
            print("이때 사용된 Target DQN W")
            print(sess.run(targetDQN._W1))
            print("이때 사용된  St+1")
            print(next_state)
            print("Target DQN의 예측값 stW 출력")
            print(Q)
        y_stack = np.vstack([y_stack, Q])
        x_stack = np.vstack([x_stack, state])
        print("Stack에 누적되는 state St")
        print(x_stack)
        print("Stack에 누적되는 target Y")
        print(y_stack)
    return mainDQN.update(x_stack, y_stack)
    
def bot_play(mainDQN):
    s = env.reset()
    reward_sum = 0
    while True:
        #env.render()
        a = np.argmax(mainDQN.predict(s))
        s, reward, done, _ = env.step(a)
        reward_sum += reward
        if done:
            print("Total score: {}".format(reward_sum))
            break
def get_copy_var_ops(*, dest_scope_name="target", src_scope_name="main"):
    op_holder = []
    src_vals = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = src_scope_name)
    dest_vals = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = dest_scope_name)
    for src_vals, dest_vals in zip(src_vals, dest_vals):
        print("메인22W")
        print(src_vals, dest_vals)
        op_holder.append(dest_vals.assign(src_vals.value()))
    return op_holder

max_episodes =9
#replay_buffer = deque()
replay_buffer = []
with tf.Session() as sess:
    mainDQN = DQN(sess, input_size, output_size, name="main")
    targetDQN = DQN(sess, input_size, output_size, name="target")
    tf.global_variables_initializer().run()
    copy_ops = get_copy_var_ops(dest_scope_name="target", src_scope_name="main")
    sess.run(copy_ops)
    print("초기네트워크 mian")
    print(sess.run(mainDQN._W1))
    print("초기네트워크 target")
    print(sess.run(targetDQN._W1))
    
    for episode in range(max_episodes):
        e = 1. / ((episode / 10) + 1)
        done = False
        step_count = 0

        state = env.reset()
        print("episode: {}".format(episode))
        print("Step마다 mianW")
        print(sess.run(mainDQN._W1))
        print("Sep 마다 targetw")
        print(sess.run(targetDQN._W1))
        while not done:
            if np.random.rand(1) < e :
                action = env.action_space.sample()
            else:
                action = np.argmax(mainDQN.predict(state))
            print(" 사용된 mian_W")
            print(sess.run(mainDQN._W1))
            print("Episode",episode)
            next_state, reward, done, _ = env.step(action)
            print("s_t: {} action: {} Reward{} ".format(state,action,reward))
            print("st+1: {}  done{}".format(next_state,done))
            #if done:
            #    reward = -100
            #REPLAY_MEMORY.append([state, action, reward, next_state, done])
            replay_buffer.append([state, action, reward, next_state, done])
            print("메모리",replay_buffer[0:1])
            print("메모리",replay_buffer[1:2])
            print("메모리",replay_buffer[2:3])
            print("메모리",replay_buffer[3:4])
            print("메모리",replay_buffer[4:5])
            print("메모리",replay_buffer[5:6])
            if len(replay_buffer) > 5:
               del replay_buffer[0]
            state = next_state
            step_count += 1
            if step_count > 10000:
                break
        print("episode: {}  step: {}".format(episode, step_count))

        if step_count > 10000:
            pass
        if episode % 4 == 3:
            for _ in range(3):
                minibatch = random.sample(replay_buffer, 2)
                loss, _ = replay_train(mainDQN, targetDQN, minibatch)
                print("episode: {}".format(episode))
                print("중간훈련후 mianDQN W")
                print(sess.run(mainDQN._W1))
                print("중간훈련후 TargetDQN W")
                print(sess.run(targetDQN._W1))
            print("Loss: ", loss)
            sess.run(copy_ops)
            print("훈련된 Wm과 새로시작할때Wt로 이동시켜 동일하게 함.Wt<=Wm",sess.run(copy_ops))
    bot_play(mainDQN)



훈련에 사용되는  self._X: St
Tensor("main/input_x:0", shape=(?, 4), dtype=float32)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


훈련함여한 W1
<tf.Variable 'main/W1:0' shape=(4, 2) dtype=float32_ref>
훈련에 사용되는  self._X: St
Tensor("target/input_x:0", shape=(?, 4), dtype=float32)
훈련함여한 W1
<tf.Variable 'target/W1:0' shape=(4, 2) dtype=float32_ref>
메인22W
<tf.Variable 'main/W1:0' shape=(4, 2) dtype=float32_ref> <tf.Variable 'target/W1:0' shape=(4, 2) dtype=float32_ref>
초기네트워크 mian


AttributeError: ignored